In [8]:
from dotenv import load_dotenv

In [9]:
load_dotenv()
import os

In [13]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

c:\Users\FAVOUR\Miniconda3\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
import json

data = json.load(open("reviews.json"))

In [3]:
from openai import OpenAI

In [1]:
import json

data = json.load(open("reviews.json"))

data['reviews']

[{'professor': 'Dr. Emily Thompson',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Dr. Thompson explains complex concepts clearly. Her enthusiasm for physics is contagious.'},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': "Brilliant instructor. Prof. Chen's projects are challenging but incredibly rewarding."},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Biology',
  'stars': 3,
  'review': 'Dr. Johnson knows her stuff, but her lectures can be a bit disorganized.'},
 {'professor': 'Prof. David Rodriguez',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Prof. Rodriguez makes calculus understandable. Always willing to help during office hours.'},
 {'professor': 'Dr. Lisa Patel',
  'subject': 'Chemistry',
  'stars': 5,
  'review': "Best chemistry professor ever! Dr. Patel's labs are well-designed and her explanations are crystal clear."},
 {'professor': 'Prof. James Wilson',
  'subject': 'History',
  'stars': 2,
  'review':

In [10]:
processed_data = []

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)
for review in data['reviews']:
    response = client.embeddings.create(
       input=review['review'],
       model="text-embedding-3-small",
    )

    embedding = response.data[0].embedding
    processed_data.append({
        "id": review["professor"],
        "values": embedding,
        "metadata": {
            "review": review["review"], 
            "stars": review["stars"], 
            "subject": review["subject"]
        }
    })

In [11]:
processed_data[0]

{'id': 'Dr. Emily Thompson',
 'values': [0.022751438,
  -0.010468082,
  -0.061907575,
  0.043136295,
  0.027807312,
  0.028130027,
  0.0006378672,
  0.017292168,
  -0.002653998,
  -0.009923499,
  0.01721149,
  0.04356658,
  -0.0070257834,
  -0.047224022,
  0.019524282,
  0.026072716,
  0.0024859172,
  -0.01827376,
  0.026812272,
  0.034127153,
  0.023706136,
  -0.022079114,
  0.021258878,
  -0.00095806137,
  -0.052172326,
  -0.063037075,
  0.047896348,
  0.03342794,
  -0.021299217,
  -0.018515797,
  0.077505484,
  -0.024620498,
  0.017345954,
  -0.03627859,
  -0.017386293,
  0.050262924,
  -0.03170679,
  0.013930549,
  0.0013765829,
  0.0008656169,
  0.010279831,
  0.017883813,
  -0.033401046,
  0.026139949,
  0.00073367334,
  -0.017910706,
  -0.057013057,
  -0.00843094,
  0.029044388,
  0.0206,
  0.0018909107,
  -0.00048701453,
  0.010542037,
  0.0045281006,
  -0.081216715,
  0.05948721,
  0.030093212,
  0.02414987,
  -0.0011286635,
  -0.05631384,
  0.018125849,
  -0.017910706,
  0.00

In [14]:
index=pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace='rateprof'
)

{'upserted_count': 20}

In [15]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'rateprof': {'vector_count': 20}},
 'total_vector_count': 20}